In [69]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

In [36]:
  from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [37]:
url="/content/drive/MyDrive/Predicting Molecular Properties/"

In [38]:
train=pd.read_csv(url+"train.csv")
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [39]:
structures=pd.read_csv(url+'structures.csv')
structures.head()

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [40]:
test=pd.read_csv(url+'test22.csv')
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type
0,4658147,dsgdb9nsd_000004,2,0,2JHC
1,4658148,dsgdb9nsd_000004,2,1,1JHC
2,4658149,dsgdb9nsd_000004,2,3,3JHH
3,4658150,dsgdb9nsd_000004,3,0,1JHC
4,4658151,dsgdb9nsd_000004,3,1,2JHC


In [41]:
t1=train.merge(structures,left_on=['molecule_name','atom_index_0'],right_on=['molecule_name','atom_index'])
t1.drop(['atom','atom_index'],axis=1,inplace=True)
t1.rename(columns={'x':'x0','y':'y0','z':'z0'},inplace=True)
t1.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,x0,y0,z0
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,0.002150,-0.006031,0.001976
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,0.002150,-0.006031,0.001976
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,0.002150,-0.006031,0.001976
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,0.002150,-0.006031,0.001976
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,1.011731,1.463751,0.000277


In [42]:
t2=t1.merge(structures,left_on=['molecule_name','atom_index_1'],right_on=['molecule_name','atom_index'])
t2.drop(['atom','atom_index'],axis=1,inplace=True)
t2.rename(columns={'x':'x1','y':'y1','z':'z1'},inplace=True)
t2.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,x0,y0,z0,x1,y1,z1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001
1,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001
2,7,dsgdb9nsd_000001,3,0,1JHC,84.8093,-0.540815,1.447527,-0.876644,-0.012698,1.085804,0.008001
3,9,dsgdb9nsd_000001,4,0,1JHC,84.8095,-0.523814,1.437933,0.906397,-0.012698,1.085804,0.008001
4,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277


In [43]:
t3=t2.merge(structures,left_on=['molecule_name','atom_index_0'],right_on=['molecule_name','atom_index'],)
t3.drop(['atom_index','x','y','z'],axis=1,inplace=True)
t3.rename(columns={'atom':'atom0'},inplace=True)
t3.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,x0,y0,z0,x1,y1,z1,atom0
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,H
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,H
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644,H
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397,H
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,H


In [44]:
t4=t3.merge(structures,left_on=['molecule_name','atom_index_1'],right_on=['molecule_name','atom_index'])
t4.drop(['atom_index','x','y','z'],axis=1,inplace=True)
t4.rename(columns={'atom':'atom1'},inplace=True)
full_dataset=t4
t4.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,x0,y0,z0,x1,y1,z1,atom0,atom1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,H,C
1,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,H,C
2,7,dsgdb9nsd_000001,3,0,1JHC,84.8093,-0.540815,1.447527,-0.876644,-0.012698,1.085804,0.008001,H,C
3,9,dsgdb9nsd_000001,4,0,1JHC,84.8095,-0.523814,1.437933,0.906397,-0.012698,1.085804,0.008001,H,C
4,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,H,H


In [45]:
del t1,t2,t3,t4

In [46]:
t1=test.merge(structures,left_on=['molecule_name','atom_index_0'],right_on=['molecule_name','atom_index'],how='left')
t1.drop(['atom','atom_index'],axis=1,inplace=True)
t1.rename(columns={'x':'x0','y':'y0','z':'z0'},inplace=True)

t2=t1.merge(structures,left_on=['molecule_name','atom_index_1'],right_on=['molecule_name','atom_index'],how='left')
t2.drop(['atom','atom_index'],axis=1,inplace=True)
t2.rename(columns={'x':'x1','y':'y1','z':'z1'},inplace=True)

t3=t2.merge(structures,left_on=['molecule_name','atom_index_0'],right_on=['molecule_name','atom_index'],how='left')
t3.drop(['atom_index','x','y','z'],axis=1,inplace=True)
t3.rename(columns={'atom':'atom0'},inplace=True)

t4=t3.merge(structures,left_on=['molecule_name','atom_index_1'],right_on=['molecule_name','atom_index'],how='left')
t4.drop(['atom_index','x','y','z'],axis=1,inplace=True)
t4.rename(columns={'atom':'atom1'},inplace=True)
full_test=t4
t4.head()

,id,molecule_name,atom_index_0,atom_index_1,type,x0,y0,z0,x1,y1,z1,atom0,atom1
0,4658147,dsgdb9nsd_000004,2,0,2JHC,-1.661639,0.0,1.0,0.599539,0.0,1.0,H,C
1,4658148,dsgdb9nsd_000004,2,1,1JHC,-1.661639,0.0,1.0,-0.599539,0.0,1.0,H,C
2,4658149,dsgdb9nsd_000004,2,3,3JHH,-1.661639,0.0,1.0,1.661639,0.0,1.0,H,H
3,4658150,dsgdb9nsd_000004,3,0,1JHC,1.661639,0.0,1.0,0.599539,0.0,1.0,H,C
4,4658151,dsgdb9nsd_000004,3,1,2JHC,1.661639,0.0,1.0,-0.599539,0.0,1.0,H,C


In [47]:
del t1,t2,t3,t4

In [48]:
full_dataset.dropna(inplace=True)
full_test.fillna(0,inplace=True)

In [49]:
full_dataset.drop('atom0',axis=1,inplace=True)
full_test.drop('atom0',axis=1,inplace=True)

In [50]:
le=LabelEncoder()
le.fit(full_dataset['type'])
full_dataset['type']=le.transform(full_dataset['type'])
full_dataset.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,x0,y0,z0,x1,y1,z1,atom1
0,0,dsgdb9nsd_000001,1,0,0,84.8076,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,C
1,4,dsgdb9nsd_000001,2,0,0,84.8074,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,C
2,7,dsgdb9nsd_000001,3,0,0,84.8093,-0.540815,1.447527,-0.876644,-0.012698,1.085804,0.008001,C
3,9,dsgdb9nsd_000001,4,0,0,84.8095,-0.523814,1.437933,0.906397,-0.012698,1.085804,0.008001,C
4,1,dsgdb9nsd_000001,1,2,3,-11.2570,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,H


In [51]:
full_test['type']=le.transform(full_test['type'])
full_test.head()

,id,molecule_name,atom_index_0,atom_index_1,type,x0,y0,z0,x1,y1,z1,atom1
0,4658147,dsgdb9nsd_000004,2,0,2,-1.661639,0.0,1.0,0.599539,0.0,1.0,C
1,4658148,dsgdb9nsd_000004,2,1,0,-1.661639,0.0,1.0,-0.599539,0.0,1.0,C
2,4658149,dsgdb9nsd_000004,2,3,6,-1.661639,0.0,1.0,1.661639,0.0,1.0,H
3,4658150,dsgdb9nsd_000004,3,0,0,1.661639,0.0,1.0,0.599539,0.0,1.0,C
4,4658151,dsgdb9nsd_000004,3,1,2,1.661639,0.0,1.0,-0.599539,0.0,1.0,C


In [52]:
full_dataset.molecule_name=[float(x[-6:]) for x in full_dataset.molecule_name]
full_test.molecule_name=[float(x[-6:]) for x in full_test.molecule_name]

In [53]:
le=LabelEncoder()
le.fit(full_dataset.atom1)
full_dataset.atom1=le.transform(full_dataset.atom1)
full_test.atom1=le.transform(full_test.atom1)

In [54]:
full_dataset.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,x0,y0,z0,x1,y1,z1,atom1
0,0,1.0,1,0,0,84.8076,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,0
1,4,1.0,2,0,0,84.8074,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,0
2,7,1.0,3,0,0,84.8093,-0.540815,1.447527,-0.876644,-0.012698,1.085804,0.008001,0
3,9,1.0,4,0,0,84.8095,-0.523814,1.437933,0.906397,-0.012698,1.085804,0.008001,0
4,1,1.0,1,2,3,-11.2570,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,1


In [55]:
"""SC=StandardScaler()
SC.fit(full_dataset.scalar_coupling_constant.values.reshape(-1,1))
full_dataset.scalar_coupling_constant=SC.transform(full_dataset.scalar_coupling_constant.values.reshape(-1,1))
"""

'SC=StandardScaler()\nSC.fit(full_dataset.scalar_coupling_constant.values.reshape(-1,1))\nfull_dataset.scalar_coupling_constant=SC.transform(full_dataset.scalar_coupling_constant.values.reshape(-1,1))\n'

In [56]:
full_dataset.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,x0,y0,z0,x1,y1,z1,atom1
0,0,1.0,1,0,0,84.8076,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,0
1,4,1.0,2,0,0,84.8074,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,0
2,7,1.0,3,0,0,84.8093,-0.540815,1.447527,-0.876644,-0.012698,1.085804,0.008001,0
3,9,1.0,4,0,0,84.8095,-0.523814,1.437933,0.906397,-0.012698,1.085804,0.008001,0
4,1,1.0,1,2,3,-11.2570,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,1


In [57]:
full_test.head()

,id,molecule_name,atom_index_0,atom_index_1,type,x0,y0,z0,x1,y1,z1,atom1
0,4658147,4.0,2,0,2,-1.661639,0.0,1.0,0.599539,0.0,1.0,0
1,4658148,4.0,2,1,0,-1.661639,0.0,1.0,-0.599539,0.0,1.0,0
2,4658149,4.0,2,3,6,-1.661639,0.0,1.0,1.661639,0.0,1.0,1
3,4658150,4.0,3,0,0,1.661639,0.0,1.0,0.599539,0.0,1.0,0
4,4658151,4.0,3,1,2,1.661639,0.0,1.0,-0.599539,0.0,1.0,0


In [58]:
x=full_dataset.drop(['scalar_coupling_constant','id'],axis=1)


In [59]:
model=tf.keras.models.Sequential([
      tf.keras.layers.Dense(11,input_dim=x.shape[1],activation='relu',kernel_initializer='normal'),
      tf.keras.layers.Dense(55,activation='relu',kernel_initializer='normal'),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(1,activation='linear',kernel_initializer='normal'),
                   
])
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mean_absolute_error'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 11)                132       
_________________________________________________________________
dense_4 (Dense)              (None, 55)                660       
_________________________________________________________________
dropout_1 (Dropout)          (None, 55)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 56        
Total params: 848
Trainable params: 848
Non-trainable params: 0
_________________________________________________________________


In [60]:
x=full_dataset.drop(['scalar_coupling_constant','id'],axis=1).values
y=full_dataset.scalar_coupling_constant.values
#history=model.fit(x,y,epochs=3,batch_size=30)

In [61]:
x_test=full_test.drop('id',axis=1)
test=model.predict(x_test)

In [62]:
from sklearn.linear_model import LinearRegression
rfr=LinearRegression()
rfr.fit(x,y)
test=rfr.predict(x_test)
test[-10:]

array([ 51.34267666,  31.31767401,  -9.33043581, -19.59607235,
        -3.69322272,  30.81858523,  30.68229269,  -5.26716475,
        27.87545967,  51.50373082])

In [63]:
test=pd.DataFrame(test)
test=pd.concat([full_test.id,test],axis=1)
test.columns=['id','scalar_coupling_constant']
test.tail()

,id,scalar_coupling_constant
2505537,7163684,30.818585
2505538,7163685,30.682293
2505539,7163686,-5.267165
2505540,7163687,27.875460
2505541,7163688,51.503731


In [64]:
test.to_csv('/content/drive/MyDrive/Predicting Molecular Properties/Submission.csv',index=False)

In [65]:
def modeling(x,y,x_test,mol):
  mod=mol
  mod.fit(x,y)
  pred=mod.predict(x_test)
  x_test.id,pd.Series(pred)
  return pd.concat([x_test.id,pd.Series(pred)],axis=1)

In [71]:
def crossval(model):
  test_chunks=[full_dataset.iloc[:full_dataset.shape[0] // 5,:] ,full_dataset.iloc[full_dataset.shape[0] // 5+1:full_dataset.shape[0] // 5 *2,:],full_dataset.iloc[full_dataset.shape[0] // 5 *2+1 :full_dataset.shape[0] // 5 *3,:],full_dataset.iloc[full_dataset.shape[0] // 5 *3+1:full_dataset.shape[0] // 5 *4,:],full_dataset.iloc[full_dataset.shape[0] // 5 *4+1:,:]]
  result=pd.DataFrame([])
  for i in test_chunks:
    test_part=i
    train_part=pd.concat(test_chunks.remove(i))
    x=train_part.drop(['scalar_coupling_constant'],axis=1).values
    y=train_part.scalar_coupling_constant.values
    x_test=test_part.drop(['scalar_coupling_constant'],axis=1).values
    pred=modeling(x,y,x_test,model)
    result=pd.concat([result,pred])
  return result

In [73]:
level_1_models=[LinearRegression(),SVR()]
level_1_pred=pd.DataFrame([])
for i in level_1_models:
  single_pred=crossval(i)
  level_1_pred=pd.contact([level_1_pred,single_pred],axis=1)